# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howrah,22.5892,88.3103,30.00,54,40,3.09,IN,1708332003
1,1,albany,42.6001,-73.9662,-0.52,78,69,1.79,US,1708331748
2,2,bethel,41.3712,-73.4140,0.27,62,0,2.57,US,1708332003
3,3,potosi,-19.5836,-65.7531,7.23,92,100,0.90,BO,1708332003
4,4,iqaluit,63.7506,-68.5145,-32.15,68,0,10.29,CA,1708332004


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Humidity"] > 39) & (city_data_df["Humidity"] < 61)]

# Drop any rows with null values
ideal_weather_df_copy = ideal_weather_df.copy()
ideal_weather_df_copy.dropna(inplace=True)

# Display sample data
ideal_weather_df_copy.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howrah,22.5892,88.3103,30.00,54,40,3.09,IN,1708332003
9,9,wailua homesteads,22.0669,-159.3780,21.44,55,100,9.26,US,1708332005
19,19,erenhot,43.6475,111.9767,-15.29,49,8,2.72,CN,1708332008
46,46,ismailia,30.6043,32.2723,17.04,51,0,4.63,EG,1708332015
52,52,port lincoln,-34.7333,135.8667,24.02,55,77,5.27,AU,1708331778


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_copy[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

C:\Users\necol\AppData\Local\Temp\ipykernel_5600\2857209497.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
0,howrah,IN,22.5892,88.3103,54,
9,wailua homesteads,US,22.0669,-159.3780,55,
19,erenhot,CN,43.6475,111.9767,49,
46,ismailia,EG,30.6043,32.2723,51,
52,port lincoln,AU,-34.7333,135.8667,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
latitude = []
longitude = []

categories = "accommodation.hotel"

radius = 10000
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = resp.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
howrah - nearest hotel: Sun India Guest House
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
erenhot - nearest hotel: No hotel found
ismailia - nearest hotel: Mercure
port lincoln - nearest hotel: Boston Hotel
phan rang-thap cham - nearest hotel: Khách Sạn Ninh Thuận
bredasdorp - nearest hotel: Victoria Hotel
miandrivazo - nearest hotel: Baobab
vangaindrano - nearest hotel: Fanilo
tazacorte - nearest hotel: App Leyma
cabo san lucas - nearest hotel: Comfort Rooms
sur - nearest hotel: Sur Hotel
margaret river - nearest hotel: Margaret River Hotel
ha'il - nearest hotel: فندق كلاسيك
salalah - nearest hotel: Muscat International Hotel
kapa'a - nearest hotel: Pono Kai Resort
alto barinas - nearest hotel: Hotel Turístico Varyná
shenyang - nearest hotel: Bestay Hotel Express Shenyang Imperial Palace Huaiyuanmen Subway Station
korets - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
hermanus - nearest hotel: Aloe guest house
saldanha - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,howrah,IN,22.5892,88.3103,54,Sun India Guest House
9,wailua homesteads,US,22.0669,-159.3780,55,Hilton Garden Inn Kauai Wailua Bay
19,erenhot,CN,43.6475,111.9767,49,No hotel found
46,ismailia,EG,30.6043,32.2723,51,Mercure
52,port lincoln,AU,-34.7333,135.8667,55,Boston Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Country",
    hover_cols = "Hotel Name"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)